# **Installing new libraries**

In [ ]:
!pip install python-dotenv
!pip install pinecone-client
!pip install openai
!pip install langchain
!pip install langchain-openai
!pip install langchain_pinecone
!pip install langchain[docarray]
!pip install pypdf
!!pip install docarray
!pip install pydantic==1.10.8
!pip install pytube
!pip install tiktoken
!pip install ruff

# **Importing libraries**

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.prompts import PromptTemplate
from operator import itemgetter
import pinecone
import tiktoken
import json
import requests
from cryptography.fernet import Fernet
from google.colab import drive
drive.mount("/content/drive", force_remount=True)
os.environ['GDRIVE_CONFIG_DIR'] = "/content/drive/MyDrive"

Mounted at /content/drive


# **Accessing the Openai and Pinecone keys**

In [ ]:
!rm -rf .env
!touch .env
!cat /content/drive/MyDrive/openai_api_key.txt >> .env
!cat /content/drive/MyDrive/pinecone_api_key.txt >> .env

In [ ]:
if not os.path.exists(".env"):
    print("Error: .env file not found. Please create a .env file and set the OPENAI_API_KEY environment variable.")
    exit(1)

# **Setting up access keys**

In [ ]:
MODEL = "gpt-3.5-turbo"
#MODEL = "mixtral:8x7b"
#MODEL = "llama2"

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
if MODEL.startswith("gpt"):
    model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

AIMessage(content="Why don't scientists trust atoms?\n\nBecause they make up everything!", response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 11, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f568d661-8a9e-48de-9475-cf19901bb34f-0')

In [ ]:
parser = StrOutputParser()
chain = model | parser
chain.invoke("Tell me a joke")

"Why couldn't the bicycle find its way home?\n\nBecause it lost its bearings!"

In [ ]:
template = """
Answer the question based on the context below. If you can't
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [ ]:
chain = prompt | model | parser
chain.invoke({"context": "My parents named me Santiago", "question": "What's your name'?"})

'My name is Santiago.'

In [ ]:
loader = PyPDFLoader("DCST925.pdf")
pages = loader.load_and_split()
pages

[Document(page_content='If you have questions or comments, contact us.  \nPour toute question ou tout commentaire, nous contacter.  \nSi tiene dudas o comentarios, contáctenos.\n1-800-4-D eWALTInstruction Manual  \nGuide D’utilisation\nManual de instruccionesDCST925\n20V Max* Lithium String Trimmer  \nTaille-bordure au lithium 20\xa0V Max*\nPodadora de cuerda de Litio 20V Max*\nfinal page size: 8.5 x 5.5 in', metadata={'source': 'DCST925.pdf', 'page': 0}),
 Document(page_content='English\nEnglish (original instructions) 1\nFrançais (traduction de la notice d’instructions originale) 13\nEspañol (traducido de las instrucciones originales) 26', metadata={'source': 'DCST925.pdf', 'page': 1}),
 Document(page_content='English\n1English (original instructions)\nDefinitions: Safety Alert Symbols and Words\nThis instruction manual uses the following safety alert symbols and words to alert you to hazardous situations and your risk \nof personal injury or property\xa0damage. \x02 DANGER:  Indicat

In [ ]:
vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()
retriever.invoke("intended use")

[Document(page_content='qui peut affecter son utilisation. Un protecteur ou une \nautre pièce qui est endommagé doit être convenablement \nréparé ou remplacé par un centre de services autorisé sauf \nsi c’est indiqué ailleurs dans ce\xa0manuel.\n15 . Dommages à l’outil : si vous frappez ou si vous vous \nemmêlé avec un objet étranger, arrêtez immédiatement \nl’outil, retirez la pile, vérifiez les dommages et faites réparer \nles dommages avant de tenter de le\xa0réutiliser.\n16 . Protecteur : ne pas utiliser cet outil sans avoir fixé le \nprotecteur. Maintenez les protecteurs en place en bon état \nde\xa0fonctionnement.\n17 . NE PAS  immerger l’outil dans l’eau ou l’asperger avec un \ntuyau. NE PAS  laisser pénétrer tout liquide à l’intérieur. \nSi l’outil est mouillé, laissez-le sécher pendant au moins \n48\xa0heures.\n18 . NE PAS  entreposer l’outil sur ou près de fertilisants ou de \nproduits\xa0chimiques.\n19 . NE PAS  nettoyer avec une laveuse à\xa0pression.\n20 . NE PAS  charger 

In [ ]:
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [ ]:
questions = [
    "What is the purpose of the tool?",
    "How many parts are there in the tool?",
    "What is the customer service number?",
    "What are the warnings you should be aware of?",
    "How do you mount it?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: What is the purpose of the tool?
Answer: The purpose of the tool is for professional trimming applications, specifically for use as a string trimmer.

Question: How many parts are there in the tool?
Answer: I don't know.

Question: What is the customer service number?
Answer: 1-800-4-DeWALT (1-800-433-9258)

Question: What are the warnings you should be aware of?
Answer: Some of the warnings you should be aware of include:
- Read all safety warnings and instructions
- Use the appliance in safe environments
- Keep bystanders away
- Dress properly
- Always wear eye protection
- Use the right appliance for the intended job
- Avoid accidentally starting the appliance
- Do not force the appliance
- Do not overreach
- Stay alert
- Disconnect the appliance when not in use
- Store idle appliances indoors
- Maintain the appliance with care
- Check damaged parts before further use
- Do not immerse the appliance in water or squirt it with a hose
- Do not store the appliance near fertili

In [ ]:
for s in chain.stream({"question": "What is the purpose of the tool?"}):
    print(s, end="", flush=True)

The purpose of the tool is for professional trimming applications. It is designed to be a string trimmer, not an edger, and is intended for cutting and trimming tasks.